pip install opencv-python
pip install imutils
pip install dlib

In [1]:
from imutils.face_utils import FaceAligner, rect_to_bb
import imutils
import dlib
import cv2
import matplotlib.pyplot as plt
from glob import glob
import os
import requests

from PIL import Image, ImageOps
from pillow_heif import register_heif_opener

register_heif_opener()

In [7]:
# capture issues to resolve in the future:
# buttons as faces
# other people
capture_issues = ['20210614_183300.jpg', '20210829_093342.jpg',
                  '20220301_222916.jpg','20220626_135456.jpg', 
                  '20220917_163316.jpg', '20220925_190928.jpg',
                  '20221128_194535.jpg']



need_to_be_renamed = [
               '327766_2110888256203_2015606625_o.jpg',
               '13483308_10206732714060887_9206378074821374390_o.jpg',
               '13938075_10207037654724213_7700935497558972159_o.jpg',
                'IMG_2267.JPG',
                'AirBrush_20210330203813.jpg']

image_issues = capture_issues + need_to_be_renamed

In [3]:
from imutils.face_utils.helpers import FACIAL_LANDMARKS_68_IDXS
from imutils.face_utils.helpers import FACIAL_LANDMARKS_5_IDXS
from imutils.face_utils.helpers import shape_to_np
import numpy as np
import cv2

class FaceAligner:
	def __init__(self, predictor, desiredLeftEye=(0.35, 0.35),
		desiredFaceWidth=256, desiredFaceHeight=None):
		# store the facial landmark predictor, desired output left
		# eye position, and desired output face width + height
		self.predictor = predictor
		self.desiredLeftEye = desiredLeftEye
		self.desiredFaceWidth = desiredFaceWidth
		self.desiredFaceHeight = desiredFaceHeight

		# if the desired face height is None, set it to be the
		# desired face width (normal behavior)
		if self.desiredFaceHeight is None:
			self.desiredFaceHeight = self.desiredFaceWidth

	def align(self, image, gray, rect):
		# convert the landmark (x, y)-coordinates to a NumPy array
		shape = self.predictor(gray, rect)
		shape = shape_to_np(shape)
		
		#simple hack ;)
		if (len(shape)==68):
			# extract the left and right eye (x, y)-coordinates
			(lStart, lEnd) = FACIAL_LANDMARKS_68_IDXS["left_eye"]
			(rStart, rEnd) = FACIAL_LANDMARKS_68_IDXS["right_eye"]
		else:
			(lStart, lEnd) = FACIAL_LANDMARKS_5_IDXS["left_eye"]
			(rStart, rEnd) = FACIAL_LANDMARKS_5_IDXS["right_eye"]
			
		leftEyePts = shape[lStart:lEnd]
		rightEyePts = shape[rStart:rEnd]

		# compute the center of mass for each eye
		leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
		rightEyeCenter = rightEyePts.mean(axis=0).astype("int")

		# compute the angle between the eye centroids
		dY = rightEyeCenter[1] - leftEyeCenter[1]
		dX = rightEyeCenter[0] - leftEyeCenter[0]
		angle = np.degrees(np.arctan2(dY, dX)) - 180

		# compute the desired right eye x-coordinate based on the
		# desired x-coordinate of the left eye
		desiredRightEyeX = 1.0 - self.desiredLeftEye[0]

		# determine the scale of the new resulting image by taking
		# the ratio of the distance between eyes in the *current*
		# image to the ratio of distance between eyes in the
		# *desired* image
		dist = np.sqrt((dX ** 2) + (dY ** 2))
		desiredDist = (desiredRightEyeX - self.desiredLeftEye[0])
		desiredDist *= self.desiredFaceWidth
		scale = desiredDist / dist

		# compute center (x, y)-coordinates (i.e., the median point)
		# between the two eyes in the input image
		eyesCenter = (int((leftEyeCenter[0] + rightEyeCenter[0]) // 2),
            int((leftEyeCenter[1] + rightEyeCenter[1]) // 2))

		# grab the rotation matrix for rotating and scaling the face
		M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

		# update the translation component of the matrix
		tX = self.desiredFaceWidth * 0.5
		tY = self.desiredFaceHeight * self.desiredLeftEye[1]
		M[0, 2] += (tX - eyesCenter[0])
		M[1, 2] += (tY - eyesCenter[1])

		# apply the affine transformation
		(w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
		output = cv2.warpAffine(image, M, (w, h),
			flags=cv2.INTER_CUBIC)

		# return the aligned face
		return output

In [4]:
landmark_name = "shape_predictor_68_face_landmarks.dat"
if not os.path.isfile(landmark_name):
    landmarks_url = "https://github.com/italojs/facial-landmarks-recognition/raw/master/shape_predictor_68_face_landmarks.dat"
    out = requests.get(landmarks_url)
    open(landmark_name, "wb").write(out.content)


In [27]:
paths = glob('original_faces/*')
failed_paths = []
verbose = False
save = True

size = 1024

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
fa = FaceAligner(predictor, desiredFaceWidth=size, desiredLeftEye=(.42,.5))

def to_aligned_path(image_path):
    aligned_base = f'aligned_faces/{os.path.basename(image_path)}'
    aligned_path =  os.path.splitext(aligned_base)[0] + '.jpg'
    return aligned_path

for image_path in paths:
    aligned_path = to_aligned_path(image_path)
    if os.path.isfile(aligned_path) or  os.path.basename(image_path) in image_issues:
        continue
    try:
        # load the input image, resize it, and convert it to grayscale
        # load and transform image
        image = Image.open(image_path).convert('RGB')
        image = ImageOps.exif_transpose(image)
        image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        image = imutils.resize(image, width=800)
        print(f'successful load of {image_path}')
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # show the original input image and detect faces in the grayscale
        # image
        cv2.imshow("Input", image)
        rects = detector(gray, 2)



        # loop over the face detections
        rect = rects[0]
        # extract the ROI of the *original* face, then align the face
        # using facial landmarks
        (x, y, w, h) = rect_to_bb(rect)
        faceOrig = imutils.resize(image[y:y + h, x:x + w], width=size)
        faceAligned = fa.align(image, gray, rect)
        if save:
            cv2.imwrite(aligned_path, faceAligned)
        # display the output images
        if verbose:
            plt.subplot(1,2,1)
            plt.imshow(faceOrig)
            plt.subplot(1,2,2)
            plt.imshow(faceAligned)
            plt.show()
    except Exception as e:
        print(e)
        failed_paths.append(image_path)


[Errno 2] No such file or directory: 'original_faces/IMG_0133.HEIC'
[Errno 2] No such file or directory: 'original_faces/cfdc965cee-result-20210822214218..HEIC'
[Errno 2] No such file or directory: 'original_faces/20210112_233814.HEIC'
[Errno 2] No such file or directory: 'original_faces/20201101_092008.HEIC'
[Errno 2] No such file or directory: 'original_faces/20201221_105043.HEIC'
[Errno 2] No such file or directory: 'original_faces/20210112_233814.HEIC'
[Errno 2] No such file or directory: 'original_faces/20210325_165212.HEIC'
[Errno 2] No such file or directory: 'original_faces/20210423_222337.HEIC'
[Errno 2] No such file or directory: 'original_faces/20210502_173447.HEIC'
[Errno 2] No such file or directory: 'original_faces/20210522_123528.HEIC'
[Errno 2] No such file or directory: 'original_faces/20210614_183300.HEIC'
[Errno 2] No such file or directory: 'original_faces/20210629_184214.HEIC'
[Errno 2] No such file or directory: 'original_faces/20210804_182126.HEIC'
[Errno 2] No s

In [104]:
def get_date_taken(path):
    exif = Image.open(path).getexif()
    if not exif:
        print('Image {0} does not have EXIF data.'.format(path))
        return None
    return exif.get(306)

dates = []
for path in paths:
    if os.path.basename(path) in image_issues:
        continue
    date = get_date_taken(path)
    if date is not None:
        dates.append((date, to_aligned_path(path)))


In [105]:
images = [i[1] for i in sorted(dates, key = lambda x: x[0])]
frame = cv2.imread(images[0])
height, width, layers = frame.shape


In [106]:
output_filename = 'aligned_faces_video.mp4'
fps = 9
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video = cv2.VideoWriter(output_filename, fourcc, fps, (width,height))
for image in images:
    video.write(cv2.imread(image))

cv2.destroyAllWindows()
video.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
[ WARN:0@2705.058] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('aligned_faces/DSC_0492.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2705.803] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('aligned_faces/20201031_184422.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2706.232] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('aligned_faces/20210111_152931.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2706.232] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modu